In [1]:
from carbon.config import *
from carbon.bot import *
from time import sleep
from itertools import combinations
from carbon.helpers import RouteStruct

# session.rollback()

bot = CarbonBot(
    # drop_tables=True
)
# bot.db.seed_pools()

ConstantProductCurve v2.6.1 (18/Apr/2023)
ArbGraph v2.1 (16/Apr/2023)
TokenScale v1.0 (07/Apr/2022)
CPCArbOptimizer v3.4.1 (19/Apr/2023)
(sqlite3.IntegrityError) UNIQUE constraint failed: blockchains.name
[SQL: INSERT INTO blockchains (name, block_number) VALUES (?, ?)]
[parameters: ('Ethereum', 17067501)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [2]:
def delete_all_carbon():
    pool = session.query(Pool).filter(Pool.exchange_name=='carbon_v1').all()
    while len(pool) > 0:
        session.delete(pool[0])
        pool = session.query(Pool).filter(Pool.exchange_name=='carbon_v1').all()
    session.commit()
    
# session.rollback()
delete_all_carbon()

In [3]:
delete_all_carbon()

In [4]:
bot.db.update_pools()

updating bancor v3 pool


2023-04-20 14:29:55,295 - fastlane - WARNING - 'NoneType' object is not a mapping


updating bancor v3 pool


2023-04-20 14:29:56,628 - fastlane - WARNING - 'NoneType' object is not a mapping


updating bancor v3 pool


2023-04-20 14:29:57,662 - fastlane - WARNING - execution reverted: execution reverted


KeyboardInterrupt: 

In [5]:
curves = []
while len(curves) < 50:
    # session.rollback()
    pools = (
        session.query(Pool)
        .filter(
            (Pool.tkn0_balance > 0)
            | (Pool.tkn1_balance > 0)
            | (Pool.liquidity > 0)
            | (Pool.y_0 > 0)
        )
        .all()
    )
    print(len(pools))
    curves = []
    for p in pools:
        try:
            curves += p.to_cpc("float")
            sleep(0.00000001) #### waiting increases the stability
        except:
            pass
    print(len(curves))
    len(CPCContainer(curves))
    CCm = CPCContainer(curves)

81
79


In [6]:
# flashloan_tokens = ["T.ETH", "T.BNT", "T.USDC", "T.USDT", "T.DAI", "T.WBTC"]
# token_combos = list(combinations(flashloan_tokens, 2))
# trade_instructions_dic = []
# for tkn0, tkn1 in token_combos:
#     # print(tkn0, tkn1)
#     # try:
#     #     pstart = {
#     #         tkn0: CCm.bypairs(f"{tkn0}/{tkn1}")[0].p
#     #     }
#     #     print(CCm.bypairs(f"{tkn0}/{tkn1}")[0].p)
#     # except:
#     #     pass

#     try:
#         CC = CCm.bypairs(f"{tkn0}/{tkn1}")
#         O = CPCArbOptimizer(CC)
#         r = O.margp_optimizer(tkn1,
#                             # params=dict(pstart=pstart, verbose=True)
#                             )
#         trade_instructions_df = r.trade_instructions(O.TIF_DFAGGR)
#         trade_instructions_dic = r.trade_instructions(O.TIF_DICTS)
#         if len(trade_instructions_dic) > 1:
#             print(r.result, tkn0, len(trade_instructions_dic))
#     except:
#         continue

In [7]:
tkn0 = T.BNT
tkn1 = T.LINK
print(CCm.bypairs(f"{tkn0}/{tkn1}")[0].p)
# pstart = CCm.bypairs(f"{tkn0}/{tkn1}")[0].p
CC = CCm.bypairs(f"{tkn0}/{tkn1}")
O = CPCArbOptimizer(CC)
r = O.margp_optimizer(tkn1,
                    # params=dict(pstart=pstart, verbose=True)
                    )
trade_instructions_df = r.trade_instructions(O.TIF_DFAGGR)
trade_instructions_dic = r.trade_instructions(O.TIF_DICTS)
print(r, tkn1)
trade_instructions_dic

0.06590029493184478
CPCArbOptimizer.MargpOptimizerResult(result=-62.101945479365895, time=0.0, method='margp', targettkn='LINK-86CA', p_optimal_t=(0.06598421081143253,), dtokens_t=(-1.973551115952432e-05,), tokens_t=('BNT-FF1C',), errormsg=None) LINK-86CA


[{'cid': '1701411834604692317316873037158841057305',
  'tknin': 'LINK-86CA',
  'amtin': 616.0274191938806,
  'tknout': 'BNT-FF1C',
  'amtout': -9341.924442499876},
 {'cid': '2722258935367507707706996859454145691668-0',
  'tknin': 'BNT-FF1C',
  'amtin': 91.88032050154288,
  'tknout': 'LINK-86CA',
  'amtout': -6.068184502146323},
 {'cid': '2722258935367507707706996859454145691669-0',
  'tknin': 'BNT-FF1C',
  'amtin': 9250.044102262822,
  'tknout': 'LINK-86CA',
  'amtout': -672.0611801711002}]

In [8]:
carbon_cid = [x['cid'].split('-')[0] for x in trade_instructions_dic if '-' in x['cid']]
carbon_cid

['2722258935367507707706996859454145691668',
 '2722258935367507707706996859454145691669']

In [9]:
session.rollback()
best_src_token = r.targettkn
src_address = session.query(Token).filter(Token.key==best_src_token).first().address
src_address = w3.toChecksumAddress(src_address)

In [10]:
def agg_carbon_independentIDs(trade_instructions):
        listti = []
        for instr in trade_instructions:

            listti += [
                {'cid': instr['cid'],
                 'tknin': instr['tknin'],
                 'amtin': instr['amtin'],
                 'tknout': instr['tknout'],
                 'amtout': instr['amtout']}
            ]
        df = pd.DataFrame.from_dict(listti)
        carbons = df[df.cid.str.contains("-")].copy()
        nocarbons = df[~df.cid.str.contains("-")]
        dropindexes = []
        new_trade_instructions = []
        carbons['pair_sorting'] = carbons.tknin + carbons.tknout
        for pair_sorting in carbons.pair_sorting.unique():
            newdf = carbons[carbons.pair_sorting==pair_sorting]
            newoutput = {
                'pair_sorting': pair_sorting,
                'cid': newdf.cid.values[0],
                'tknin': newdf.tknin.values[0],
                'amtin': newdf.sum()['amtin'],
                'tknout': newdf.tknout.values[0],
                'amtout': newdf.sum()['amtout'],
                'raw_txs': str(newdf.to_dict(orient='records'))
            }
            new_trade_instructions.append(newoutput)

        print("new_trade_instructions", new_trade_instructions)
        nocarbons_instructions = []
        dictnocarbons =  nocarbons.to_dict(orient='records')
        for dict in dictnocarbons:
            dict['pair_sorting'] = dict['tknin']+dict['tknout']
            dict['raw_txs'] = str([])
            nocarbons_instructions += [dict]

        new_trade_instructions += nocarbons_instructions
        trade_instructions = [TradeInstruction(**new_trade_instructions[i]) for i in range(len(new_trade_instructions))]
        return trade_instructions

In [11]:
session.rollback()
trade_instructions = r.trade_instructions(O.TIF_DICTS)

tx_route_handler = TxRouteHandler(trade_instructions)
agg_trade_instructions = agg_carbon_independentIDs(trade_instructions=trade_instructions)

# TODO: cleanup
new_trade_instructions = []
if len(agg_trade_instructions) == 2:
    for inst in agg_trade_instructions:
        if inst.tknin == best_src_token:
            new_trade_instructions += [inst]
    missing = [inst for inst in agg_trade_instructions if inst not in new_trade_instructions]
    new_trade_instructions.append(missing[0])
else:
    new_trade_instructions = tx_route_handler._find_tradematches(agg_trade_instructions)

trade_instructions = new_trade_instructions

count = 0
for inst in trade_instructions:
    if (inst.cid == carbon_cid) & (count == 0):
        print('Carbon FIRST')
    else:
        print('Carbon LAST')
    count +=1

src_amount = trade_instructions[0].amtin_wei
print(f"src_token: {best_src_token}")
print(f"src_amount: {src_amount}")

# trade_instructions = tx_route_handler._calculate_trade_outputs(
#     trade_instructions
# )
src_address = session.query(Token).filter(Token.key==best_src_token).first().address
src_address = w3.toChecksumAddress(src_address)

trade_instructions = tx_route_handler.custom_data_encoder(trade_instructions)

tx_submit_handler = TxSubmitHandler(trade_instructions,
                                    src_amount=trade_instructions[0].amtin_wei,
                                    src_address=src_address)
deadline = tx_submit_handler._get_deadline()

route_struct = tx_route_handler.get_arb_contract_args(
    trade_instructions, deadline
)

new_trade_instructions [{'pair_sorting': 'BNT-FF1CLINK-86CA', 'cid': '2722258935367507707706996859454145691668-0', 'tknin': 'BNT-FF1C', 'amtin': 9341.924422764365, 'tknout': 'LINK-86CA', 'amtout': -678.1293646732465, 'raw_txs': "[{'cid': '2722258935367507707706996859454145691668-0', 'tknin': 'BNT-FF1C', 'amtin': 91.88032050154288, 'tknout': 'LINK-86CA', 'amtout': -6.068184502146323, 'pair_sorting': 'BNT-FF1CLINK-86CA'}, {'cid': '2722258935367507707706996859454145691669-0', 'tknin': 'BNT-FF1C', 'amtin': 9250.044102262822, 'tknout': 'LINK-86CA', 'amtout': -672.0611801711002, 'pair_sorting': 'BNT-FF1CLINK-86CA'}]"}]
Carbon LAST
Carbon LAST
src_token: LINK-86CA
src_amount: 616027419193880608768
DETAILS 0 616.0274191938806 -9341.924442499876
DETAILS 1 9341.924422764365 -678.1293646732465
trade_instruction.cid: 1701411834604692317316873037158841057305
trade_instruction.cid: 2722258935367507707706996859454145691668


In [12]:
route_struct

[RouteStruct(exchangeId=2, targetToken='0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', minTargetAmount=1, deadline=1681971429, customAddress='0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', customInt=0, customData=''),
 RouteStruct(exchangeId=6, targetToken='0x514910771AF9Ca656af840dff83E8264EcF986CA', minTargetAmount=1, deadline=1681971429, customAddress='0x514910771AF9Ca656af840dff83E8264EcF986CA', customInt=0, customData=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\xeeX:\x0b\xe2\x11\xaf\xd3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00

In [14]:
# b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\xeeX:\x0b\xe2\x11\xaf\xd3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x15\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xf0nr\xa1&\xf7\xd9\xbb\x16'.hex()
# 0000000000000000000000000000000000000000000000000000000000000020
# 0000000000000000000000000000000000000000000000000000000000000002
# 0000000000000000000000000000000800000000000000000000000000000014
# 000000000000000000000000000000000000000000000004ee583a0be211afd3
# 0000000000000000000000000000000800000000000000000000000000000015
# 0000000000000000000000000000000000000000000001f06e72a126f7d9bb16

'000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000800000000000000000000000000000014000000000000000000000000000000000000000000000004ee583a0be211afd300000000000000000000000000000008000000000000000000000000000000150000000000000000000000000000000000000000000001f06e72a126f7d9bb16'

In [15]:
tx_details = tx_submit_handler._get_tx_details()
tx_submit_handler.token_contract.functions.approve(
    w3.toChecksumAddress(FASTLANE_CONTRACT_ADDRESS), 0
).transact(tx_details)
tx_submit_handler.token_contract.functions.approve(
    w3.toChecksumAddress(FASTLANE_CONTRACT_ADDRESS), src_amount
).transact(tx_details)

tx = tx_submit_handler._submit_transaction_tenderly(
    route_struct, src_address, src_amount
)

w3.eth.wait_for_transaction_receipt(tx)

{'exchangeId': 2, 'targetToken': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'minTargetAmount': 1, 'deadline': 1681971429, 'customAddress': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'customInt': 0, 'customData': ''}


{'exchangeId': 6, 'targetToken': '0x514910771AF9Ca656af840dff83E8264EcF986CA', 'minTargetAmount': 1, 'deadline': 1681971429, 'customAddress': '0x514910771AF9Ca656af840dff83E8264EcF986CA', 'customInt': 0, 'customData': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\xeeX:\x0b\xe2\x11\xaf\xd3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

AttributeDict({'transactionHash': HexBytes('0x1dfcf4b1041cc505b0c368e75a8a84a4b07399e8364bcc6ad7dbaba2ebafccb2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x5cd8ce171dc2a6649981bef07a7792f82a505d677ccb93651e33112ebb1ae225'),
 'blockNumber': 17067504,
 'from': '0x28C6c06298d514Db089934071355E5743bf21d60',
 'to': '0x41Eeba3355d7D6FF628B7982F3F9D055c39488cB',
 'cumulativeGasUsed': 467434,
 'gasUsed': 467434,
 'effectiveGasPrice': 0,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'address': '0x514910771AF9Ca656af840dff83E8264EcF986CA',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000649765821d9f64198c905ec0b2b037a4a52bc373'),
    HexBytes('0x00000000000000000000000041eeba3355d7d6ff628b7982f3f9d055c39488cb')],
   'data': '0x000000000000000000000000000000000000000000000021651909e757e80000',
   'blockHash': HexBytes('0x5cd8ce171dc2a6649981bef07a7792f82a505d677ccb93651e33112ebb1ae